Let us begin with intalling dependencies

In [ ]:
!pip install yt-dlp opencv-python-headless ultralytics supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.8/874.8 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.1 MB/s eta 0:00:00


We need to convert the traffic video inputs from youtube to a API for being processed by the program

In [ ]:
import yt_dlp

# Replace with your YouTube video URLs
YOUTUBE_VIDEO_URL1 = "https://youtu.be/iJZcjZD0fw0?si=ipq66LnIp4-0Hv2a"
YOUTUBE_VIDEO_URL2 = "https://youtu.be/MNn9qKG2UFI?si=W4cklEsDahsk2p4p"
YOUTUBE_VIDEO_URL3 = "https://youtu.be/UVCMOQyIz6A?si=98yfZkzIv1aHGuM0"
YOUTUBE_VIDEO_URL4 = "https://youtu.be/UVCMOQyIz6A?si=98yfZkzIv1aHGuM0"

def get_best_video_url(youtube_url):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'quiet': True,
        'noplaylist': True,
        'skip_download': True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=False)

        # Print the full info_dict for debugging
        print("Info Dictionary:", info_dict)

        # Extract the best format URL
        formats = info_dict.get('formats', [])
        for format in formats:
            if format.get('ext') == 'mp4' and format.get('format_id') == '18':  # You can adjust format_id or conditions as needed
                return format.get('url')
        return None

# Get video URLs
video_url1 = get_best_video_url(YOUTUBE_VIDEO_URL1)
video_url2 = get_best_video_url(YOUTUBE_VIDEO_URL2)
video_url3 = get_best_video_url(YOUTUBE_VIDEO_URL3)
video_url4 = get_best_video_url(YOUTUBE_VIDEO_URL4)
print(video_url1)
print(video_url2)
print(video_url3)
print(video_url4)

Info Dictionary: {'id': 'iJZcjZD0fw0', 'title': '15 minutes of heavy traffic noise in India | 14-08-2022', 'formats': [{'format_id': 'sb3', 'format_note': 'storyboard', 'ext': 'mhtml', 'protocol': 'mhtml', 'acodec': 'none', 'vcodec': 'none', 'url': 'https://i.ytimg.com/sb/iJZcjZD0fw0/storyboard3_L0/default.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgjB-8mnBg==&sigh=rs$AOn4CLAjGkbRAj826KeR9LrBSC24H08cdw', 'width': 48, 'height': 27, 'fps': 0.1111111111111111, 'rows': 10, 'columns': 10, 'fragments': [{'url': 'https://i.ytimg.com/sb/iJZcjZD0fw0/storyboard3_L0/default.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgjB-8mnBg==&sigh=rs$AOn4CLAjGkbRAj826KeR9LrBSC24H08cdw', 'duration': 900.0}], 'resolution': '48x27', 'aspect_ratio': 1.78, 'filesize_approx': None, 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.20 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language'

We also need to install MQTT for transmitting the decision from the server to the junction

In [ ]:
!pip install paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 4.2 MB/s eta 0:00:00


Now we have completed all the pre-requestics for the SMART TRAFFIC SYSTEM. The program is given below where the vehicle count is taken from each lane and the opening time for the lane with more vehciles is decided and sent to the junction. The system will again process only after the calculated time and the process will continue. Thus dynammic timing is introduced in the traffic system making it the SMART TRAFFIC SYSTEM.

In [ ]:
import paho.mqtt.client as mqtt
import cv2
from ultralytics import YOLO
import supervision as sv
import threading
import time
import logging

# Suppress unwanted logs from libraries
logging.getLogger('ultralytics').setLevel(logging.ERROR)
logging.getLogger('torch').setLevel(logging.ERROR)

# MQTT Configuration
broker = "test.mosquitto.org"
port = 1883
topic = "device/traffic1"
qos = 0

# Connect to MQTT Broker
def on_connect(client, userdata, flags, rc, properties=None):
    if rc == 0:
        print("Connected to MQTT Broker!")
    else:
        print(f"Failed to connect, return code {rc}\n")

# Initialize MQTT Client
client = mqtt.Client(protocol=mqtt.MQTTv5)
client.on_connect = on_connect
client.connect(broker, port, keepalive=60)
client.loop_start()

model = YOLO("yolov8x.pt", verbose=False)

# Vehicle and lane information
vehicle_counts = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
waiting_times = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
opened_lanes = []  # Tracks which lanes have been opened

# Lane opening constants
MIN_OPEN_TIME = 10  # Minimum opening time in seconds
MAX_OPEN_TIME = 60  # Maximum time a lane can stay open

# Function to calculate opening time dynamically
def calculate_opening_time(vehicle_count):
    return min(MAX_OPEN_TIME, MIN_OPEN_TIME + vehicle_count * 2)

# Function to process a video stream for a lane
def process_video_stream(video_url, lane_id):
    cap = cv2.VideoCapture(video_url)
    if not cap.isOpened():
        print(f"[{lane_id}] Error opening video stream.")
        return

    ret, frame = cap.read()
    if not ret:
        print(f"[{lane_id}] End of video stream.")
        return

    # Process the frame
    result = model(frame, imgsz=1280)[0]
    detections = sv.Detections.from_ultralytics(result)
    detections = detections[detections.confidence > 0.3]
    detections = detections[detections.class_id != 0]  # Exclude class 0 (persons)
    vehicle_count = len(detections)
    vehicle_counts[lane_id] = vehicle_count

    print(f"[{lane_id}] Detected {vehicle_count} vehicles.")
    cap.release()

# Function to manage lane control based on priorities and dynamic timings
def manage_lane_control(video_urls):
    while True:
        print("Calculating lane priorities...")

        # Calculate lane priority scores (placeholder logic)
        lane_priority_scores = {
            'A': waiting_times['A'],
            'B': waiting_times['B'],
            'C': waiting_times['C'],
            'D': waiting_times['D'],
        }

        # Select lane with highest priority
        available_lanes = {k: v for k, v in lane_priority_scores.items() if k not in opened_lanes}
        if available_lanes:
            max_priority_lane = max(available_lanes, key=available_lanes.get)
            print(f"Lane {max_priority_lane} selected for opening.")

            # Dynamically calculate opening time for the lane
            lane_opening_time = calculate_opening_time(vehicle_counts[max_priority_lane])
            print(f"Lane {max_priority_lane} will remain open for {lane_opening_time} seconds.")

            # Publish MQTT message for lane opening
            client.publish(topic, max_priority_lane, qos=qos)
            print(f"Published MQTT message for Lane {max_priority_lane}.")

            # Process the video stream for the selected lane
            process_video_stream(video_urls[ord(max_priority_lane) - 65], max_priority_lane)

            # Wait for the lane's calculated opening time
            time.sleep(lane_opening_time)

            # Mark this lane as opened in the current cycle
            opened_lanes.append(max_priority_lane)

            # If all lanes have been opened, reset for a new cycle
            if len(opened_lanes) == 4:
                print("All lanes have been opened, resetting cycle.")
                opened_lanes.clear()

        # Sleep for a short time before rechecking priorities
        time.sleep(1)

# Function to process all video streams and control lanes
def process_videos(video_urls):
    global vehicle_counts, waiting_times
    vehicle_counts = {chr(64 + i): 0 for i in range(1, 5)}
    waiting_times = {chr(64 + i): 0 for i in range(1, 5)}

    # Start lane control management in a separate thread
    lane_control_thread = threading.Thread(target=manage_lane_control, args=(video_urls,))
    lane_control_thread.start()

    lane_control_thread.join()

    # Stop MQTT client once done
    client.loop_stop()
    client.disconnect()

# Video URLs (replace with actual video paths or URLs)
video_urls = [video_url1, video_url2, video_url3, video_url4]

# Start processing videos
process_videos(video_urls)


Connected to MQTT Broker!
Calculating lane priorities...
Lane A selected for opening.
Lane A will remain open for 10 seconds.
Published MQTT message for Lane A.
[A] Detected 29 vehicles.
Calculating lane priorities...
Lane B selected for opening.
Lane B will remain open for 10 seconds.
Published MQTT message for Lane B.
[B] Detected 8 vehicles.
Calculating lane priorities...
Lane B selected for opening.
Lane B will remain open for 26 seconds.
Published MQTT message for Lane B.
[B] Detected 8 vehicles.
Calculating lane priorities...
Lane C selected for opening.
Lane C will remain open for 10 seconds.
Published MQTT message for Lane C.
[C] Detected 19 vehicles.
All lanes have been opened, resetting cycle.
Calculating lane priorities...
Lane A selected for opening.
Lane A will remain open for 60 seconds.
Published MQTT message for Lane A.
[A] Detected 29 vehicles.
Calculating lane priorities...
Lane A selected for opening.
Lane A will remain open for 60 seconds.
Published MQTT message for

This version of the code is where the dynamic timing is not introduced but a preset time is used. This is still efficient compared to the present traffic systems.

In [ ]:
import paho.mqtt.client as mqtt
import cv2
from ultralytics import YOLO
import supervision as sv
import threading
import time
import logging

# Suppress unwanted logs from libraries
logging.getLogger('ultralytics').setLevel(logging.ERROR)
logging.getLogger('torch').setLevel(logging.ERROR)

# MQTT Configuration
broker = "test.mosquitto.org"
port = 1883
topic = "device/traffic"
qos = 0

def on_connect(client, userdata, flags, rc, properties=None):
    if rc == 0:
        print("Connected to MQTT Broker!")
    else:
        print(f"Failed to connect, return code {rc}\n")

# Callback function when a message is published
def on_publish(client, userdata, mid):
    print(f"Message {mid} published.")

# Initialize the MQTT client with new API format
client = mqtt.Client(protocol=mqtt.MQTTv5)
client.on_connect = on_connect
client.on_publish = on_publish
client.connect(broker, port, keepalive=60)
client.loop_start()

model = YOLO("yolov8x.pt", verbose=False)

# Dictionary to store waiting times and vehicle counts for each lane
waiting_times = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
vehicle_counts = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
opened_lanes = []  # To track the order of lanes that have been opened

# Priority Score Calculation Function
def calculate_priority_score(vehicle_count, waiting_time, threshold=60):
    if waiting_time > threshold:
        return vehicle_count * waiting_time * 1.5
    return vehicle_count * waiting_time

# Function to process a single video stream
def process_video_stream(video_url, lane_id):
    cap = cv2.VideoCapture(video_url)
    if not cap.isOpened():
        print(f"[{lane_id}] Error opening video stream.")
        return

    start_time = time.time()

    while True:
        current_time = time.time()

        # Check if 30 seconds have passed to process the frame
        if current_time - start_time >= 15:

            ret, frame = cap.read()
            if not ret:
                print(f"[{lane_id}] End of video stream.")
                break

            result = model(frame, imgsz=1280)[0]
            detections = sv.Detections.from_ultralytics(result)
            detections = detections[detections.confidence > 0.3]
            detections = detections[detections.class_id != 0]
            vehicle_count = len(detections)
            vehicle_counts[lane_id] = vehicle_count

            # Increment waiting time
            waiting_times[lane_id] += 30  # Increment by 30 seconds

            # Reset the timer after processing
            start_time = time.time()

        # Sleep for a short time to avoid high CPU usage
        time.sleep(1)

    cap.release()

# Function to handle publishing of messages every 30 seconds
def publish_messages():
    last_publish_time = time.time()

    while True:
        current_time = time.time()

        # Check if 30 seconds have passed since the last publishing
        if current_time - last_publish_time >= 15:
            print("Publishing messages...")

            # Print vehicle counts for each lane
            for lane_id in vehicle_counts:
                print(f"Lane {lane_id} has {vehicle_counts[lane_id]} vehicles.")

            # Calculate priority scores
            lane_priority_scores = {
                'A': calculate_priority_score(vehicle_counts['A'], waiting_times['A']),
                'B': calculate_priority_score(vehicle_counts['B'], waiting_times['B']),
                'C': calculate_priority_score(vehicle_counts['C'], waiting_times['C']),
                'D': calculate_priority_score(vehicle_counts['D'], waiting_times['D']),
            }

            # Filter lanes that have not been opened in the current cycle
            available_lanes = {k: v for k, v in lane_priority_scores.items() if k not in opened_lanes}

            if available_lanes:
                # Find the lane with the highest priority score from available lanes
                max_priority_lane = max(available_lanes, key=available_lanes.get)
                message = {'A': 'a', 'B': 'b', 'C': 'c', 'D': 'd'}.get(max_priority_lane, 'unknown')
                client.publish(topic, message, qos=qos)
                print(f"Published message '{message}' for Lane {max_priority_lane}.")

                # Mark this lane as opened
                opened_lanes.append(max_priority_lane)

                # If all lanes have been opened, reset the opened_lanes list
                if len(opened_lanes) == 4:
                    print("All lanes have been opened, resetting cycle.")
                    opened_lanes.clear()

            # Reset waiting times after publishing
            for lane in waiting_times:
                waiting_times[lane] = 0

            last_publish_time = current_time

        # Sleep for a short time to avoid high CPU usage
        time.sleep(1)

# Function to process multiple video streams in parallel
def process_videos(video_urls):
    # Initialize vehicle counts and waiting times
    global vehicle_counts, waiting_times
    vehicle_counts = {chr(64 + i): 0 for i in range(1, 5)}
    waiting_times = {chr(64 + i): 0 for i in range(1, 5)}

    # Start processing threads for each video stream
    threads = []
    for i, video_url in enumerate(video_urls, start=1):
        lane_id = chr(64 + i)  # 'A', 'B', 'C', 'D'
        thread = threading.Thread(target=process_video_stream, args=(video_url, lane_id))
        threads.append(thread)
        thread.start()

    # Start publishing thread
    publishing_thread = threading.Thread(target=publish_messages)
    publishing_thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    publishing_thread.join()

    # Cleanup MQTT connection
    client.loop_stop()
    client.disconnect()

# Video URLs (replace these with actual video paths or URLs)
video_urls = [video_url1, video_url2, video_url3, video_url4]

# Process videos
process_videos(video_urls)